In [1]:
#!pip install plotly
#!pip install plotly-geo
#!pip install pyshp
#!pip install geopandas


In [2]:
import os
import sys

repo = 'microbusiness-density-forecasting'
ix = os.getcwd().index(repo)
ROOT_PATH = os.path.join(os.getcwd()[:ix], repo)
DATA_PATH = os.path.join(ROOT_PATH, 'data/raw')
    
    
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.figure_factory as ff
import geopandas
import shapely

In [3]:
census_df = pd.read_csv(os.path.join(DATA_PATH,'census_starter.csv'))
sample_df = pd.read_csv(os.path.join(DATA_PATH,'sample_submission.csv'))
train_df = pd.read_csv(os.path.join(DATA_PATH,'train.csv'))
test_df = pd.read_csv(os.path.join(DATA_PATH,'test.csv'))

In [4]:
train_df['dataset'] = 'train'
test_df['dataset'] = 'test'
df = pd.concat((train_df, test_df)).sort_values('row_id').reset_index(drop=True)

In [5]:
df.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,dataset
0,10001_2019-08-01,10001,Kent County,Delaware,2019-08-01,4.995701,6624.0,train
1,10001_2019-09-01,10001,Kent County,Delaware,2019-09-01,5.031902,6672.0,train
2,10001_2019-10-01,10001,Kent County,Delaware,2019-10-01,5.098270,6760.0,train
3,10001_2019-11-01,10001,Kent County,Delaware,2019-11-01,5.124666,6795.0,train
4,10001_2019-12-01,10001,Kent County,Delaware,2019-12-01,5.086203,6744.0,train


In [6]:
df_all = df.merge(census_df, on = 'cfips', how='left')

In [7]:
df_all['first_day_of_month'] = pd.to_datetime(df_all["first_day_of_month"])
df_all["year"] = df_all["first_day_of_month"].dt.year
df_all['month'] = df_all["first_day_of_month"].dt.month

In [8]:
fig = ff.create_choropleth(fips=census_df.cfips,title = 'pct_bb_2019', values=census_df.pct_bb_2019.values)
fig.layout.template = None
fig.show();

/Users/austingeary/opt/anaconda3/lib/python3.9/site-packages/plotly/figure_factory/_county_choropleth.py:775: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  fips_polygon_map[f].type
/Users/austingeary/opt/anaconda3/lib/python3.9/site-packages/plotly/figure_factory/_county_choropleth.py:331: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if fips_polygon_map[f].type == "Polygon":
/Users/austingeary/opt/anaconda3/lib/python3.9/site-packages/plotly/figure_factory/_county_choropleth.py:358: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif fips_polygon_map[f].type == "MultiPolygon":


TypeError: 'MultiPolygon' object is not iterable

In [ ]:
#add year and month columns
def target_shape(input_df):
    df = input_df.copy(deep=True)
    df['year']=df['first_day_of_month'].str.split('-',expand=True).iloc[:,0]
    df['month']=df['first_day_of_month'].str.split('-',expand=True).iloc[:,1]
    df['year_month']=df['year']+df['month']
    df['year']=df['year'].astype('int')
    df['month']=df['month'].astype('int')
    df = df.drop('first_day_of_month', axis=1)
    df['cfips_year'] = df['cfips'].astype(str) + '_' + df['year'].astype(str)
    return df
df2 = target_shape(df)

In [ ]:
census_df.head()

In [ ]:
def census_shape(input_df):
    df = input_df.copy(deep=True)
    df = pd.wide_to_long(df,
                         stubnames=['pct_bb','pct_college','pct_foreign_born','pct_it_workers', 'median_hh_inc'], 
                         i='cfips', j='year', sep='_', suffix='\d+')
    df.sort_index(axis=0, level=[0,1], ascending=True, inplace=True)
    df.reset_index(inplace=True)
    dfs = []
    for cfip in df['cfips'].unique():
        county_df = df[df['cfips'] == cfip]
        county_df.interpolate(method='linear', limit_direction='both', inplace=True)
        dfs.append(county_df)
    df = pd.concat(dfs)
    df['cfips_year'] = df['cfips'].astype(str) + '_' + df['year'].astype(str)
    return df
census_df2 = census_shape(census_df)

In [ ]:
census_df2.head()

In [ ]:
train_df2.head()

In [ ]:
test_df.head()

In [ ]:
plt.plot(train_df[train_df['cfips'] == 1001]['first_day_of_month'],
         train_df[train_df['cfips'] == 1001]['microbusiness_density'] )
plt.xticks(rotation=90)
plt.show()